In [5]:
from propositional_state_logic import *
from utils import *
from collections import defaultdict
import itertools
from copy import deepcopy

In [6]:
p = Problem()

# Define the variables above. Returns a Variable object.
F1 = p.add_variable('F1', type='binary')
P1 = p.add_variable('P1', type='binary')
V1 = p.add_variable('V1', type='binary')
P2 = p.add_variable('P2', type='binary')
V2 = p.add_variable('V2', type='binary')
PV2 = p.add_variable('PV2', type='binary')
V3 = p.add_variable('V3', type='binary')
PV3 = p.add_variable('PV3', type='binary')
P3 = p.add_variable('P3', type='binary')
R1 = p.add_variable('R1', type='binary')
T1 = p.add_variable('T1', type='binary')

# Add the word problem constraints.
p.add_constraint('F1 => P1')
p.add_constraint('~F1 => ~P1')

p.add_constraint('V1 & P1 => P2')
p.add_constraint('V1 & ~P1 => ~P2')
p.add_constraint('~V1 => ~P2')

p.add_constraint('V2 & P2 => PV2')
p.add_constraint('V2 & ~P2 => ~PV2')
p.add_constraint('~V2 => ~PV2')

p.add_constraint('V3 & P2 => PV3')
p.add_constraint('V3 & ~P2 => ~PV3')
p.add_constraint('~V3 => ~PV3')

p.add_constraint('~PV2 & ~PV3 => ~P3')
p.add_constraint('PV2 => P3')
p.add_constraint('PV3 => P3')

p.add_constraint('R1 & P3 => T1')
p.add_constraint('R1 & ~P3 => ~T1')
p.add_constraint('~R1 => ~T1')

# Prints out constraints nicely in LaTeX, so you can check them.
display_constraints(p)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [7]:
sat = SATSolver(p)
# Make an assignment to some variables; variable.get_assignment(value) returns an Assignment.
assignment = frozenset([F1.get_assignment('True'), P1.get_assignment('False')])
sat.check_consistency(assignment)

(False, None, frozenset())

In [8]:
Vs = ['T1', 'R1', 'P3']

In [9]:
Vs

['T1', 'R1', 'P3']

In [136]:
def findsubsets(s, n):
    return itertools.combinations(s, n)


In [138]:
nodes = []

for i in range(len(Vs)+1):
    for n in findsubsets(Vs, i):
        nodes.append(n)

In [139]:
nodes

[[],
 ['T1'],
 ['R1'],
 ['P3'],
 ['T1', 'R1'],
 ['T1', 'P3'],
 ['R1', 'P3'],
 ['T1', 'R1', 'P3']]

In [14]:
tree = {}
level = 0

In [123]:
l = {}

for n in nodes[1:]:
    for j in nodes[1:]:
        if n != j and n == j[:len(n)] and len(j) == len(n)+1:
            if n in l.keys():
                l[n].append(j)
            else:
                l[n] = [j]


In [128]:
def children(nodes, n):
    print('enter')
    print('n',n)
    print('nodes',nodes)
    if n in nodes:
        nodes.remove(n)
    print('nodes',nodes)
    
    if nodes == [] or [j for j in nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1] == []:
        return {}
    else:
        print('children', [j for j in nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1])
        a = [j for j in nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1][0]
        print('child',a)
        return {n : {j: children([k for k in nodes if k != j], j) for j in nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1}}

In [122]:
children(nodes[1:], ('T1',))

enter
n ('T1',)
nodes [('T1',), ('R1',), ('P3',), ('T1', 'R1'), ('T1', 'P3'), ('R1', 'P3'), ('T1', 'R1', 'P3')]
nodes [('R1',), ('P3',), ('T1', 'R1'), ('T1', 'P3'), ('R1', 'P3'), ('T1', 'R1', 'P3')]
children [('T1', 'R1'), ('T1', 'P3')]
child ('T1', 'R1')
enter
n ('T1', 'R1')
nodes [('R1',), ('P3',), ('T1', 'P3'), ('R1', 'P3'), ('T1', 'R1', 'P3')]
nodes [('R1',), ('P3',), ('T1', 'P3'), ('R1', 'P3'), ('T1', 'R1', 'P3')]
children [('T1', 'R1', 'P3')]
child ('T1', 'R1', 'P3')
enter
n ('T1', 'R1', 'P3')
nodes [('R1',), ('P3',), ('T1', 'P3'), ('R1', 'P3')]
nodes [('R1',), ('P3',), ('T1', 'P3'), ('R1', 'P3')]
enter
n ('T1', 'P3')
nodes [('R1',), ('P3',), ('T1', 'R1'), ('R1', 'P3'), ('T1', 'R1', 'P3')]
nodes [('R1',), ('P3',), ('T1', 'R1'), ('R1', 'P3'), ('T1', 'R1', 'P3')]


{('T1',): {('T1', 'R1'): {('T1', 'R1'): {('T1', 'R1', 'P3'): {}}},
  ('T1', 'P3'): {}}}

In [ ]:
nodes = nodes[1:]
root = ('T1',)

while len(nodes) > 0:
    print('enter')
    print('n',n)
    print('nodes',nodes)
    if n in nodes:
        nodes.remove(n)
    print('nodes',nodes)
    
    if nodes == [] or [j for j in nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1] == []:
        print({})
    else:
        print('children', [j for j in nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1])
        a = [j for j in nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1][0]
        print('child',a)
        return {n : {j: children([k for k in nodes if k != j], j) for j in nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1}}

In [220]:
nodes = []

for i in range(len(Vs)+1):
    for n in findsubsets(Vs, i):
        nodes.append(n)

print(nodes)

[[], ['T1'], ['R1'], ['P3'], ['T1', 'R1'], ['T1', 'P3'], ['R1', 'P3'], ['T1', 'R1', 'P3']]


In [221]:
def list_tree(nodes, n):
    new_nodes = deepcopy(nodes)
    if n in new_nodes:
        new_nodes.remove(n)
    
    if new_nodes == [] or [j for j in new_nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1] == []:
        n.append([])
        
    else:
        n.append([list_tree([k for k in new_nodes if k != j], j) for j in new_nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1])
    
    return n

In [225]:
list_tree(nodes[1:], ['T1'])

['T1', [['T1', 'R1', [['T1', 'R1', 'P3', []]]], ['T1', 'P3', []]]]

In [224]:
nodes

[[],
 ['T1'],
 ['R1'],
 ['P3'],
 ['T1', 'R1'],
 ['T1', 'P3'],
 ['R1', 'P3'],
 ['T1', 'R1', 'P3']]

In [204]:
nodes[1:]

[['T1'],
 ['R1'],
 ['P3'],
 ['T1', 'R1', [['T1', 'R1', 'P3', []]]],
 ['T1', 'P3', []],
 ['R1', 'P3'],
 ['T1', 'R1', 'P3', []]]